In [ ]:
#Import modules
import numpy as np
import numpy.random as rand
import matplotlib.pyplot as plt
import scipy.special as special
import scipy.integrate as integrate
import cmath

ci = complex(0, 1)

In [ ]:
def RK4(xn, yn, dt, f, *args):
    k1 = dt*f(xn, yn, *args)
    k2 = dt*f(xn + dt/2, yn + k1/2, *args)
    k3 = dt*f(xn + dt/2, yn + k2/2, *args)
    k4 = dt*f(xn + dt, yn + k3, *args)
    
    #Return x_{n+1} and y_{n+1} as new x_n and y_n
    xn = xn + dt
    yn = yn + (k1 + 2*k2 + 2*k3 + k4)/6
    return xn, yn

In [ ]:
#Electric Field
E0 = 0.0534
omega = 0.114
phi = 0

#Envelope with mean at t=0 and duration of 128 au
def envelope(t):
    return (np.cos(np.pi*t/128))**2

def E(t):
    return E0 * envelope(t) * np.cos(omega*t + phi)

In [ ]:
#Newton's Equation
def ddx(t, dx, x):
    return -E(t) - np.absolute(x)/x**3

def dx(t, x, v):
    return v

In [ ]:
#Ionization Potential
Ip = 0.5

#Keldysh Parameter
kel = omega*np.sqrt(2*Ip)/E0

#Final time
tf = 64

In [ ]:
#Initial velocity
v0 = 0
v0t = 0

In [ ]:
#Number of trials
N = 10**4

final_p = np.empty(N)
weights = np.empty(N)
phases = np.empty(N)

#Loop over number of trials
for j in range(N):

    #Choose time and find initial position
    #Step size of 0.005 au for time
    t = np.linspace(-64, 64, 25600)
    index = rand.randint(0, 25600)
    t0 = t[index]

    x = np.zeros(25600)
    x[index] = (-Ip - np.sqrt(Ip**2 - 4*np.absolute(E(t0))))/(2*E(t0))

    p = np.zeros(25600)
    p[index] = v0

    #Solving the IVP
    for i in range(index+1, 25600):
        dt = t[i] - t[i-1]
        throw, x[i] = RK4(t[i-1], x[i-1], dt, dx, p[i-1])
        throw, p[i] = RK4(t[i-1], p[i-1], dt, ddx, x[i-1])

    t = t[index:]
    x = x[index:]
    p = p[index:]
    
    #List of final momenta
    final_p[j] = p[-1]
    
    #List of weights
    weights[j] = np.exp(-2*(2*Ip)**1.5 / (3*E(t0))) * np.exp(-np.sqrt(2*Ip)*v0t**2 / (E(t0)))
    
    #List of phases
    #Parameters from Semiclassical 2-Step paper
    L = 0
    H = (p[-1])**2 / 2 - 1/np.absolute(x[-1])
    b = 1/(2*H)
    g = np.sqrt(1 + 2*H*(L**2))
    #Build answer
    answer = -v0t*x[0] + Ip*t0
    answer = answer - cmath.sqrt(b)*(np.log(g) + np.arcsinh(x[-1]*p[-1]/(g*cmath.sqrt(b))))
    integrand = p**2 / 2 - 2/np.absolute(x)
    integral = (64-t0)*np.average(integrand)
    phases[j] = answer - integral

print('done')

<ipython-input-7-86e07807bff5>:50: ComplexWarning: Casting complex values to real discards the imaginary part
  phases[j] = answer - integral
<ipython-input-7-86e07807bff5>:37: RuntimeWarning: overflow encountered in exp
  weights[j] = np.exp(-2*(2*Ip)**1.5 / (3*E(t0))) * np.exp(-np.sqrt(2*Ip)*v0t**2 / (E(t0)))


In [ ]:
bin_size = 0.01
min_p = np.min(final_p)
max_p = np.max(final_p)
bins_pos = np.empty(np.ceil(max_p/bin_size))
bins_neg = np.empty(-np.floor(min_p/bin_size))
bins_w_pos = np.empty(np.ceil(max_p/bin_size))
bins_S_pos = np.empty(np.ceil(max_p/bin_size))
bins_w_neg = np.empty(-np.floor(min_p/bin_size))
bins_S_neg = np.empty(-np.floor(min_p/bin_size))

#Allocate momenta into bins
for i in range(np.ceil(max_p/bin_size)):
    bins_pos[i] = []
    bins_w_pos[i] = []
    bins_S_pos[i] = []
    for j in range(len(final_p)):
        if final_p[j] > bin_size*i & final_p[j] < bin_size*(i+1):
            bins_pos[i].append(final_p[j])
            final_p[j] = 0
            bins_w_pos[i].append(weights[j])
            weights[j] = 0
            bins_S_pos[i].append(phases[j])
            phases[j] = 0
    final_p = final_p[final_p!=0]
    weights = weights[weights!=0]
    phases = phases[phases!=0]

for i in range(-np.floor(min_p/bin_size)):
    bins_neg[i] = []
    bins_w_neg[i] = []
    bins_S_neg[i] = []
    for j in range(len(final_p)):
        if final_p[j] < -bin_size*i & final_p[j] > -bin_size*(i+1):
            bins_neg[i].append(final_p[j])
            final_p[j] = 0
            bins_w_neg[i].append(weights[j])
            weights[j] = 0
            bins_S_neg[i].append(phases[j])
            phases[j] = 0
    final_p = final_p[final_p!=0]
    weights = weights[weights!=0]
    phases = phases[phases!=0]
         
#Concatenate lists
bins = bins_neg.reverse() + bins_pos
bins_w = bins_w_neg.reverse() + bins_w_pos
bins_S = bins_S_neg.reverse() + bins_S_pos

In [ ]:
#Probability in bins
def prob_bin(w_bin, S_bin):
    answer = 0
    for k in len(w_bin):
        answer += np.sqrt(w[k]) * np.exp(ci*S[k])
    return np.real(answer * np.conj(answer))
         
momenta = np.linspace((np.floor(min_p/bin_size)+0.5)*bin_size, (np.ceil(max_p/bin_size)+0.5)*binsize, len(bins))

prob = np.empty(len(bins))
for i in range(len(bins)):
    prob[i] = prob_bin(bins_w[i], bins_S[i])

In [ ]:
#Plot graph
fig, ax = plt.subplots(figsize=(9, 6))
ax[0].plot(momenta, prob)